# 1. 튜닝 범위 설정의 필요성

## 하이퍼 파라미터 튜닝 문제의 특징
하이퍼 파라미터 튜닝 문제는 분석적으로 풀 수 없고 푸는데 시간이 오래 걸립니다.

- 분석적으로 해결할 수 없어, 여러 해를 평가하고 비교하여 좋은 해를 찾아야 합니다.
- 탐색 공간을 모두 탐색하는 것은 비현실적이고 비효율적임
    - 라쏘의 하이퍼 파라미터는 0 이상이기만 하면 되므로  선택할 수 있는 경우의 수가 무한대임
- 하이퍼 파라미터에 대한 탐색 공간은 사용자가 직접 정의하며, 이 과정에서 모델과 하이퍼 파라미터에 대한 이해가 큰 역할을 함
    - 라쏘의 하이퍼 파라미터는 특정 수치 이상 설정할 필요가 없음
    - 결정 나무에서 불순도 지표는 탐색할 필요가 없음
- 하나의 해를 평가하려면 모델을 학습해야 하므로 오랜 시간이 걸림. 따라서 탐색 공간을 정의하는 것과 좋을 것이라 예상되는 해를 먼저 탐색하는 것이 매우 중요함

## 튜닝할 하이퍼 파라미터 선정
모든 하이퍼 파라미터를 튜닝하는 것은 현실적으로 불가능하므로 튜닝할 하이퍼 파라미터를 선택해야 합니다.
- 하이퍼 파라미터 튜닝 문제를 효율적으로 풀기 어려운 이유는 개별 해를 평가하는 데 오랜 시간이 들기도 하지만 탐색 공간 자체가 매우 넓음
    - 사이킷런에서 DecisionTreeClassifier의 하이퍼 파라미터는 12개나 되며, 몇몇 하이퍼 파라미터는 연속형 변수임
- 따라서 모든 하이퍼 파라미터를 튜닝하는 것은 현실적으로 불가능하므로 모델의 성능에 큰 영향을 끼치는 하이퍼 파라미터를 선택해야 함
    - 결정 나무에서 노드를 분지하는 기준을 나타내는 하이퍼 파라미터 criterion은 'gini'와 'entropy' 가운데 어느 값을 설정하더라도 성능 차이가 크지 않다고 알려져 있으므로 튜닝 대상에 포함할 필요가 없음

어떤 하이퍼 파라미터를 튜닝할 것인가도 중요하지만, 어느 범위에서 튜닝할 것인가도 매우 중요합니다.
- 결정 나무의 최대 깊이 : max_depth
    - 최대 깊이는 어느 수치 미만이면 과소적합됨
    - 최대 깊이는 어느 수치 이상되면 성능에 영향을 전혀 주지 못함

## 하이퍼 파라미터 및 범위 설정을 위한 실험 프로세스
후보 하이퍼파라미터에 대해 그리드 서치를 적용한 결과를 실험 데이터로 사용합니다. 실험 데이터에 따라 다른 결과가 나올 수 있으므로 실제 시스템에 적용하려면 다양한 데이터를 바탕으로 실험 데이터를 구축해야 합니다. 또한, 반복측정 분산분석을 통해 주요 하이퍼 파라미터를 식별하고 결정 나무를 이용해 적절한 하이퍼 파라미터의 범위를 찾습니다.

# 2. 주요 하이퍼 파라미터 식별

## 반복측정 분산분석이란?
반복측정 분산분석(One-way repeated measures analysis of variance; RM ANOVA)은 반복측정 자료에 대해 그룹별 평균 차이를 검정하기 위한 가설검정 기법입니다. 대응 표본 t-검정 기법을 확장한 기법이라고도 볼 수 있습니다.

## statsmodels를 이용한 반복측정 분산분석
statsmodels는 가설 검정, 회귀 분석, 시계열 분석 등 다양한 통계 분석 기능을 제공하는 파이썬 패키지입니다. statsmodels의 AnovaRM 클래스를 사용하면 손쉽게 반복측정 분산분석을 구현할 수 있습니다.

statsmodels 설치

In [ ]:
!pip install statsmodels

AnovaRM 클래스의 주요 인자

|인자|설명
|---|---|
|data|분석에 사용할 데이터프레임|
|depvar|종속 변수명 (data의 칼럼명)|
|subject|실험 대상 ID (data의 칼럼명)|
|within|요인 (data의 칼럼명 목록)|

## 반복측정 분산분석 예제
statsmodels의 AnovaRM 클래스를 사용해서 반복측정 분산분석을 구현해보겠습니다.

예제 데이터 불러오기

In [ ]:
import pandas as pd
data = pd.read_csv('AnovaRM_예제.csv')
display(data)

- 3개의 샘플에 대해 요인 x를 바꿔가며 세 개의 실험을 수행했을 때의 종족 변수 y의 값으로 구성된 데이터임
- sub_id가 1인 샘플의 y는 x가 a일 때 10, b일 때 20,
c일 때 30으로 x가 y에 영향을 끼침을 알 수 있음

AnovaRM을 사용해 x가 y에 얼마나 영향을 끼치는지 확인해보겠습니다.

AnovaRM 사용

In [ ]:
from statsmodels.stats.anova import AnovaRM
aovrm = AnovaRM(data, 'y', 'sub_id', within = ['x'])
res = aovrm.fit()
print(res)

- 라인 1: statsmodels.stats.anova에 속한 AnovaRM 클래스를 불러옵니다.
- 라인 2: AnovaRM의 객체 aovrm을 생성합니다.
- 라인 3: fit 메서드를 적용한 결과를 res에 저장합니다. statsmodels에 속한 대부분의 인스턴스가 fit를 이용해 통계 분석을 수행합니다. 또한, fit 메서드를 사용하면 AnovaResults라는 객체를 반환하므로 res라는 변수에 저장합니다.

- 행의 이름이 x라고 돼 있음
- 즉, 인자가 x 하나이므로 x가 y에 얼마나 영향을 끼치는지만 표시됨
- 결과를 해석할 때 보는 값은 일원분산분석에서 계산한 F-통계량인 F Value와 통계적으로 얼마나 유의한 결과인지를 나타내는 Pr > F (p-value)임
- 일반적응로 p-value가 0.05보다 작으면 통게적으로 유의한 영향을 끼친다고 볼 수 있음

In [ ]:
display(res.__dict__['anova_table'])

- 라인 1: __dict__ 속성을 사용해 AnovaResult를 사전 자료형으로 변환한 뒤, 'anova_table'키로 값을 가져옵니다. 이 값은 앞서 출력한 분석 결과가 데이터프레임으로 정리된 것 입니다.

이전 출력값인 문자열보다 데이터프레임이라서 사용하기 적절함

## 실험 데이터 준비 : 후보 하이퍼 파라미터 선정
LightGBM의 주요 하이퍼 파라미터를 색출하고 그 튜닝 범위를 설정하겠습니다.

하이퍼 파라미터 범위
|하이퍼 파라미터|설명|튜닝 범위|
|---|---|---|
|n_estimator|결정 나무 개수|{50, 100, 150, 200}|
|learning_rate|학습률|{0.01, 0.05, 0.1, 0.3}|
|num_leaves|최대 잎 노드 개수|{2^3, 2^4, 2^5, 2^6}|
|boosting_type|부스팅 방법|{gbdg, goss}|
|random_state|시드|{2020, 2021, 2022, 2023}|
|metric|노드 분할 기준|{mae, mse}|
|reg_alpha|L1 패널티에 대한 가중치|{0, 0.1, 1.0, 10}|
|colsample_bytree|결정 나무별 사용하는 특징의 비율|{0.5, 0.7, 0.9}|

## 실험 데이터 준비 : 데이터 준비
실험에 사용할 데이터 mortgage.csv를 불러오고 특징과 라벨로 분리합니다.

예제 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv('../data/regression/mortgage.csv')
X = df.drop('y', axis = 1)
y = df['y']

## 실험 데이터 준비 : 그리드 서치
파라미터 그리드인 grid를 생성하고 각 해를 평가합니다.

그리드 서치

In [ ]:
from lightgbm import LGBMRegressor as LGB
from sklearn.model_selection import ParameterGrid, cross_val_score
grid = ParameterGrid({  'n_estimator': [50, 100, 150, 200],
                        'learning_rate': [0.01, 0.05, 0.1, 0.3],
                        'num_leaves': [2**3, 2**4, 2**5, 2**6],
                        'boosting_type': ['gbdt', 'goss', 'rf'],
                        'random_state': [2020, 2021, 2022, 2023],
                        'metric': ['mae', 'mse'],
                        'reg_alpha': [0, 0.1, 1.0, 10],})

score_list = []
for param in grid:
    score = ( - cross_val_score(LGB(**param), X, y,
                                scoring = 'neg_mean_absolute_error')).mean()
    score_list.append(score)

grid를 데이터프레임으로 변환하고 거기에 score_list를 칼럼으로 추가하겠습니다. 또한, 그 데이터를 저장하겠습니다.

그리드 서치

In [ ]:
grid_search_data = pd.DataFrame(gird)
grid_search_data['score'] = score_list
display(grid_search_data.head())
grid_search_data.to_csv('LightGBM_하이퍼파라미터선택_실험데이터.csv', index = False)

## 주요 하이퍼 파라미터 식별 : boosting_type 예시
반복측정 분산분석을 이용해 boosting_type의 중요도를 측정해보겠습니다.

주요 하이퍼 파라미터 식별 예시

In [ ]:
hyper_params = grid_search_data.columns[:-1].tolist()
hyper_params.remove('boosting_type')
hyper_params.insert(0, 'boosting_type')
grid_search_data.sort_values(by = hyper_params, inplace = True)
boosting_type_size = len(grid_search_data['boosting_type'].unique())
num_ID = int(len(grid_search_data) / boosting_type_size)
grid_search_data['subject_ID'] = list(range(num_ID)) * boosting_type_size

- 라인 1: grid_search_data의 칼럼에서 맨 뒤 요소(score)를 제외한 모든 요소로 구성된 리스트를 hyper_params에 저장합니다.
- 라인 2~3: hyper_params에서 중요도를 측정하고 하는 하이퍼 파라미터를 맨 앞에 오도록 했습니다. 즉, remove 메서드를 사용해 'boosting_type'을 지운 뒤에 insert 메서드를 사용해 다시 맨 앞으로 추가했습니다.
- 라인 4: hyper_params를 기준으로 오름차순 정렬합니다. 정렬하지 않으면 ID가 부적절하게 부여될 수 있습니다.
- 라인 5~6: boosting_type가 갖는 유니크한 값의 개수를 바탕으로 ID 개수를 계산합니다. boosting_type 변수는 3개의 값을 가질 수 있어, 3개의 그룹에 대한 반복측정 분산분석을 적용해야 합니다. 또한, 각 그룹에 들어가는 샘플 수만큼 ID를 부여합니다.
- 라인 7: [0, 1, 2, ..., num_ID -1]을 boosting_type_size만큼 반복하여 만든 ID를 subject_ID 칼럼으로 추가합니다.

샘플 ID는 각 하이퍼 파라미터가 독립 변수이고 나머지 하이퍼 파라미터가 통제 변수가 되도록 부여해야 함

즉, 각 하이퍼 파라미터 값만 다르고 나머지 하이퍼 파라미터 값이 같으면 샘플 ID가 같게 부여해야 함

반복측정 분산분석을 이용해 boosting_type의 중요도를 측정해보겠습니다.

주요 하이퍼 파라미터 식별 예시

In [ ]:
aovrm = AnovaRM(grid_search_data, 'score', 'subject_ID', within = ['boosting_type'])
res = aovrm.fit()
res = res.__dict__['anova_table']
display(res)

- p - value가 0.0으로 boosting_type이 성능에 영향을 준다고 할 수 있음
- 그러나 데이터가 충분히 많으면 P-value가 0.0이 나오는 경우는 매우 흔하므로, P-value만 보고 매우 중요한 하이퍼 파라미터라고 결론 짓기는 이름
- 따라서 여러 하이퍼 파라미터에 대해 반복측정 분산분석을 적용하여 나온 F-통계량을 비교해야 함

## 주요 하이퍼 파라미터 식별 : 전체 데이터에 적용
전체 하이퍼 파라미터에 대해 반복측정 분산분석을 수행하겠습니다.

주요 하이퍼 파라미터 식별

In [ ]:
AnovaRM_result = pd.DataFrame() # 분석 결과 초기화
params_cols = grid_search_data.columns.tolist()

## 주요 하이퍼 파라미터 식별 : 결과 해석

# 3. 하이퍼 파라미터 범위 설정

## 결정 나무를 이용한 하이퍼 파라미터 범위 설정

## 데이터 준비

## 결정 나무 학습

## 규칙으로 변환

## 하이퍼 파라미터 범위 설정